In [33]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [34]:
import tensorflow as tf
sess = tf.InteractiveSession()

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x108050ad0>> ignored


In [35]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [18]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [19]:
# OUTPUT SCORE
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [20]:
# LOSS FUNC
cross_entropy = - tf.reduce_mean(tf.reduce_sum(y_*tf.log(y), reduction_indices = [1]))


In [23]:
# TRAINIG STEP -> repeatingly running train_step will do all the training
tf.initialize_all_variables().run()

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [27]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


NameError: name 'y' is not defined

In [ ]:
# CONVULTIONAL NETWORK

In [1]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    # fill all values with 0.1 for the biases
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [2]:
# strides: first two are the dims of the stride, third is color channel (depth), fourth is num of kernels/filter/features
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [37]:
# initialize the weight kernels/filters for the convolution
W_conv1 = weight_variable([5,5,1,32])
B_conv1 = bias_variable([32])

In [38]:
x = tf.placeholder(tf.float32, shape=[None, 784])
x_image = tf.reshape(x, [-1,28,28,1]) # whats -1? 

In [39]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + B_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [40]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [41]:
# FC LAYER
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# flatten the pooling output into a 1dim vector
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [42]:
# Create a dropout option
prob_keep = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, prob_keep)

In [43]:
# readout / classify layer 
# seems like dropout just zeros out the dropped neurons
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

In [44]:
final_y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [66]:
# Layers all set up
# What's left: set up training process 
# then run a session

# set up the loss function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(final_y), reduction_indices=[1]))

# training step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# correct_pred will be [True, False, True] etc.
correct_pred = tf.equal(tf.argmax(y_,1), tf.argmax(final_y, 1))

# cast the array + find average
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Saver obj for checkoints
saver = tf.train.Saver()
# saver.restore(sess, FLAGS.train_dir)

# this only runs the init op
sess.run(tf.initialize_all_variables())

for i in range(200):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y_:batch[1], prob_keep:1.0})
        # save checkpoints 
        saver.save(sess, FLAGS.train_dir, global_step=step)
        print("step %d, training accuracy %g"%(i, train_accuracy))
        
    train_step.run(feed_dict={x:batch[0], y_:batch[1], prob_keep: 0.5})
    
print("test accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, prob_keep:1.0}))

sess.close()
            




step 0, training accuracy 0.16
step 100, training accuracy 0.8
step 200, training accuracy 0.9
step 300, training accuracy 0.88
step 400, training accuracy 0.92
step 500, training accuracy 0.92
step 600, training accuracy 0.94
step 700, training accuracy 0.98
step 800, training accuracy 0.98
step 900, training accuracy 1
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.92
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 0.98
step 1800, training accuracy 0.98
step 1900, training accuracy 1
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 0.92
step 2300, training accuracy 0.98
step 2400, training accuracy 1
step 2500, training accuracy 0.96
step 2600, training accuracy 0.98
step 2700, training accuracy 0.94
step 2800, training accuracy 1
step 2900, training accuracy 1
step 3000, trai

KeyboardInterrupt: 

In [73]:
summary_op = tf.merge_all_summaries()
summary_writer = tf.train.SummaryWriter(mnist., sess.graph)
summary_str = sess.run(summary_op, feed_dict=feed_dict)
summary_writer.add_summary(summary_str, step)


UnicodeDecodeError: 'ascii' codec can't decode byte 0xa9 in position 808: ordinal not in range(128)